## Advanced Regression on House Rent Prediction

In [ ]:
import os
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 

In [ ]:
# loading the datasets
train_df = pd.read_csv('Data\\train_data.csv')
test_df = pd.read_csv('Data\\test_data.csv')

In [ ]:
# peeking the dataset
train_df.head()